In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [21]:
matches_df = pd.read_csv('../Datasets/feature_dataset.csv')
matches_df.head()
# matches_df.columns
# matches_df.shape

,Season,Home Team,Away Team,Home Goals,Away Goals,Home Team ELO,Away Team ELO,Home XG,Away XG,Home XGA,Away XGA,Home Win Percentage,Home Draw Percentage,Away Win Percentage,Away Draw Percentage,Home Team Form,Away Team Form,Home Team Cumulative Points,Away Team Cumulative Points,Winner
0,2014-2015,Manchester Utd,Swansea City,1,2,1676,1521,54.21,33.84,40.90,55.94,64.50,20.64,29.82,28.95,0.0,0.0,0,0,-1
1,2014-2015,West Brom,Sunderland,2,2,1485,1520,38.63,37.45,53.75,51.46,24.01,26.97,26.52,25.10,0.0,0.0,0,0,0
2,2014-2015,Leicester City,Everton,2,2,1462,1692,48.21,44.89,56.39,46.18,27.27,29.22,36.44,28.54,0.0,0.0,0,0,0
3,2014-2015,West Ham,Tottenham,0,1,1509,1680,46.69,57.40,52.39,57.04,32.51,25.29,39.62,25.83,0.0,0.0,0,0,-1
4,2014-2015,QPR,Hull City,0,1,1420,1434,45.73,65.26,32.79,46.99,30.42,24.58,21.05,26.32,0.0,0.0,0,0,-1


In [22]:
X = matches_df.drop(["Winner","Season"], axis=1)
y = matches_df["Winner"]

categorical_columns = ["Home Team", "Away Team"]
numerical_columns = [col for col in X.columns if col not in categorical_columns]

label_mapping = {-1: 0, 0: 1, 1: 2}
y_mapped = y.map(label_mapping)

ohencoder = OneHotEncoder(sparse_output=False)
encoded_categorical_columns = ohencoder.fit_transform(X[categorical_columns])

# scaler = StandardScaler()
# scaled_numerical_columns = scaler.fit_transform(X[numerical_columns])
scaled_numerical_columns = X[numerical_columns]

X = np.hstack((scaled_numerical_columns, encoded_categorical_columns))

X_train, X_test, y_train, y_test = train_test_split(X, y_mapped, test_size=0.2, random_state=42)

In [23]:
model = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32)

Epoch 1/100


c:\Users\vimal\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3616 - loss: 36.7767 - val_accuracy: 0.4684 - val_loss: 2.5493
Epoch 2/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3801 - loss: 5.4532 - val_accuracy: 0.4684 - val_loss: 1.3946
Epoch 3/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3741 - loss: 2.1097 - val_accuracy: 0.4684 - val_loss: 1.0567
Epoch 4/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 990us/step - accuracy: 0.3992 - loss: 1.2740 - val_accuracy: 0.4684 - val_loss: 1.1117
Epoch 5/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3974 - loss: 1.1868 - val_accuracy: 0.4684 - val_loss: 1.0710
Epoch 6/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4222 - loss: 1.1259 - val_accuracy: 0.4684 - val_loss: 1.0520
Epoch 7/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4305 - loss: 1.1096 - val_accuracy: 0.4684 - val_loss: 1.0554
Epoch 8/100
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4055 - loss: 1.1190 - val_accuracy: 0.4684 - val_loss: 

In [24]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 729us/step - accuracy: 0.9798 - loss: 0.0880
Test Loss: 0.0852065160870552
Test Accuracy: 0.9842105507850647
